In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import os
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

/Users/19563044/Documents/giga/gigachain_telegram_bot/venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
import time

index_name = "gigachain-test-index-gpt-2"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [4]:
# from langchain_community.embeddings.gigachat import GigaChatEmbeddings
# embeddings = GigaChatEmbeddings(model="EmbeddingsGigaR")
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [5]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
import os
import tqdm
markdown_path = "mdx_docs/"

docuemnts = []
# Iterate all mdx file in subdirectories
for root, dirs, files in tqdm.tqdm(os.walk(markdown_path)):
    for file in files:
        if file.endswith(".mdx") or file.endswith(".md"):
            loader = UnstructuredMarkdownLoader(os.path.join(root, file))
            docuemnts.append(loader.load())
print(f"Total documents: {len(docuemnts)}")

19it [00:03,  6.26it/s]

Total documents: 53


In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs_list = [item for sublist in docuemnts for item in sublist]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=16000, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs_list)
print(f"Documents splited. Count: {len(doc_splits)}")

Documents splited. Count: 56


In [16]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(doc_splits))]
vector_store.add_documents(documents=doc_splits, ids=uuids)
retriever = vector_store.as_retriever()
print("OK")

OK
